In [1]:
import numpy as np
import theano
import theano.tensor as tt
import hky
from importlib import reload
reload(hky)

from hky import *

## Testing gradients

In [64]:
Pt = hky_transition_probs(1.0, 2.0, np.ones(4)/4, 1).eval()
Pt

array([[0.3953989 , 0.17226874, 0.21616618, 0.21616618],
       [0.17226874, 0.3953989 , 0.21616618, 0.21616618],
       [0.21616618, 0.21616618, 0.3953989 , 0.17226874],
       [0.21616618, 0.21616618, 0.17226874, 0.3953989 ]])

In [22]:
x, y = np.array([1, 0]), np.array([2, 2])
Pt[x, y].eval()

array([0.86466472, 0.86466472])

In [38]:
Pt_expm = hky_transition_probs_expm(1.0, 2.0, np.ones(4)/4, 1).eval()
Pt_expm

array([[0.3953989 , 0.17226874, 0.21616618, 0.21616618],
       [0.17226874, 0.3953989 , 0.21616618, 0.21616618],
       [0.21616618, 0.21616618, 0.3953989 , 0.17226874],
       [0.21616618, 0.21616618, 0.17226874, 0.3953989 ]])

In [101]:
import numdifftools as nd
import pandas as pd

def test_transition_prob_grad(func, point, i, j):
    theta = tt.vector()
    alpha = theta[0]
    beta = theta[1]
    pi = theta[2:6]
    t = theta[6]
    Pt_ij = func(alpha, beta, pi, t)[i, j]
    f = theano.function([theta], Pt_ij)
    Pt_ij_grad = tt.grad(Pt_ij, theta)
    f_grad = theano.function([theta], Pt_ij_grad)
    return {
        'val': f(point),
        'theano_grad': f_grad(point),
        'num_grad': nd.Gradient(f)(theta_point)
    }

theta_point = np.array([1.0, 2.0, 0.2, 0.25, 0.3, 0.25, 1.0])
pd.DataFrame(test_transition_prob_grad(hky_transition_probs_expm, theta_point, 1, 2)).transpose()

,0,1,2,3,4,5,6
num_grad,-5.292850e-16,0.033834,-0.148499,0.716166,-0.148499,-0.148499,0.067668
theano_grad,6.245005e-17,0.033834,-0.135022,0.702690,-0.135022,-0.161975,0.067668
val,2.161662e-01,0.216166,0.216166,0.216166,0.216166,0.216166,0.216166


In [102]:
pd.DataFrame(test_transition_prob_grad(hky_transition_probs, theta_point, 1, 2)).transpose()

,0,1,2,3,4,5,6
num_grad,0.000000,0.033834,0.000000,0.864665,0.000000,0.000000,0.067668
theano_grad,0.000000,0.033834,0.000000,0.864665,0.000000,0.000000,0.067668
val,0.216166,0.216166,0.216166,0.216166,0.216166,0.216166,0.216166


## Testing vectorisation

In [5]:
def hky_transition_probs_scalar(kappa, pi, t):
	piY = pi[T] + pi[C]
	piR = pi[A] + pi[G]
	lambd = make_tensor([ # Eigenvalues 
		0,
		-1,
		-(piY + piR*kappa),
		-(piY*kappa + piR)		
	])
	U = make_tensor([ # Right eigenvectors as columns (rows of transpose)
		[1, 1, 1, 1],
		[1/piY, 1/piY, -1/piR, -1/piR],
		[0, 0, pi[G]/piR, -pi[A]/piR],
		[pi[C]/piY, -pi[T]/piY, 0, 0]
	]).transpose()

	Vt = make_tensor([ # Left eigenvectors as rows
		[pi[T], pi[C], pi[A], pi[G]],
		[piR*pi[T], piR*pi[C], -piY*pi[A], -piY*pi[G]],
		[0, 0, 1, -1],
		[1, -1, 0, 0]
	])

	Q_unnormalised = tt.dot(U, tt.dot(tt.diag(lambd), Vt))
	average_subs = -tt.nlinalg.trace(tt.dot(Q_unnormalised, tt.diag(pi)))

	return tt.dot(U, tt.dot(tt.diag(tt.exp(lambd / average_subs * t)), Vt))

In [25]:
pi = np.array([0.2, 0.3, 0.18, 0.32])
kappa = 1.2
ts = np.array([2.3, 3.2, 0.1, 5.8])

In [26]:
kappa_ = tt.scalar()
pi_ = tt.vector()
t_scalar = tt.scalar()
P_scalar = hky_transition_probs_scalar(kappa_, pi_, t_scalar)

scalar_func = theano.function([kappa_, pi_, t_scalar], P_scalar)
res_scalar = [scalar_func(kappa, pi, t) for t in ts]
res_scalar

[array([[0.23981613, 0.17510156, 0.28310435, 0.30197797],
        [0.19455729, 0.2203604 , 0.28310435, 0.30197797],
        [0.18873623, 0.16986261, 0.33070206, 0.3106991 ],
        [0.18873623, 0.16986261, 0.29128041, 0.35012075]]),
 array([[0.21234738, 0.17898108, 0.29451849, 0.31415305],
        [0.19886786, 0.1924606 , 0.29451849, 0.31415305],
        [0.19634566, 0.17671109, 0.30910078, 0.31784247],
        [0.19634566, 0.17671109, 0.29797732, 0.32896593]]),
 array([[0.90199005, 0.02511698, 0.0352708 , 0.03762218],
        [0.02790775, 0.89919927, 0.0352708 , 0.03762218],
        [0.02351386, 0.02116248, 0.91069229, 0.04463137],
        [0.02351386, 0.02116248, 0.04184191, 0.91348175]]),
 array([[0.20042372, 0.18001468, 0.29978787, 0.31977373],
        [0.20001631, 0.18042209, 0.29978787, 0.31977373],
        [0.19985858, 0.17987272, 0.30027846, 0.31999024],
        [0.19985858, 0.17987272, 0.29999085, 0.32027785]])]

In [27]:
t_vector = tt.vector()
P_vector = hky_transition_probs(kappa_, pi_, t_vector)
vector_func = theano.function([kappa_, pi_, t_vector], P_vector)
res_vector = vector_func(kappa, pi, ts)
res_vector, res_vector.shape

(array([[[0.23981613, 0.21234738, 0.90199005, 0.20042372],
         [0.17510156, 0.17898108, 0.02511698, 0.18001468],
         [0.28310435, 0.29451849, 0.0352708 , 0.29978787],
         [0.30197797, 0.31415305, 0.03762218, 0.31977373]],
 
        [[0.19455729, 0.19886786, 0.02790775, 0.20001631],
         [0.2203604 , 0.1924606 , 0.89919927, 0.18042209],
         [0.28310435, 0.29451849, 0.0352708 , 0.29978787],
         [0.30197797, 0.31415305, 0.03762218, 0.31977373]],
 
        [[0.18873623, 0.19634566, 0.02351386, 0.19985858],
         [0.16986261, 0.17671109, 0.02116248, 0.17987272],
         [0.33070206, 0.30910078, 0.91069229, 0.30027846],
         [0.3106991 , 0.31784247, 0.04463137, 0.31999024]],
 
        [[0.18873623, 0.19634566, 0.02351386, 0.19985858],
         [0.16986261, 0.17671109, 0.02116248, 0.17987272],
         [0.29128041, 0.29797732, 0.04184191, 0.29999085],
         [0.35012075, 0.32896593, 0.91348175, 0.32027785]]]), (4, 4, 4))

In [28]:
res_vector[:, :, 0]

array([[0.23981613, 0.17510156, 0.28310435, 0.30197797],
       [0.19455729, 0.2203604 , 0.28310435, 0.30197797],
       [0.18873623, 0.16986261, 0.33070206, 0.3106991 ],
       [0.18873623, 0.16986261, 0.29128041, 0.35012075]])

In [29]:
[np.allclose(res_scalar[i], res_vector[:, :, i]) for i in range(len(ts))]

[True, True, True, True]

In [11]:
import pruning
from importlib import reload
reload(pruning)

theano.config.optimizer = 'fast_compile'

from pruning import *

leaf_sequences_ = tt.matrix(dtype = 'int64')
leaf_sequences = np.array([[0, 1, 2, 1, 1], [0, 1, 1, 1, 3], [1, 1, 3, 1, 2], [0, 1, 3, 0, 0]])

child_indices_ = tt.matrix(dtype = 'int64')
child_indices = np.array([[0, 1], [2, 3], [0, 1]])

child_transition_probs_ = tt.tensor4()
p_un = np.arange(1, 3 * 2 * 4 * 4 + 1).reshape(3, 2, 4, 4)
child_transition_probs = p_un/p_un.sum(axis = 2, keepdims=True)

leaf_parent_mask_ = tt.vector(dtype = 'bool')
leaf_parent_mask = np.array([True, True, False])

partials_ = phylogenetic_likelihood_math(child_indices_, child_transition_probs_, leaf_parent_mask_, leaf_sequences_)

partials_func = theano.function([child_indices_, child_transition_probs_, leaf_parent_mask_, leaf_sequences_], partials_[0])
partials = partials_func(child_indices, child_transition_probs, leaf_parent_mask, leaf_sequences)
partials

array([[[[0.00659938, 0.04076087, 0.08734472, 0.14635094],
         [0.01171875, 0.04296875, 0.08463541, 0.13671875],
         [0.015625  , 0.04456019, 0.08275463, 0.13020833],
         [0.01171875, 0.04296875, 0.08463541, 0.13671875],
         [0.01201923, 0.04326923, 0.08413462, 0.13461539]],

        [[0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ]],

        [[0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        , 0.        ]]],


       [[[0.00659938, 0.04076087, 0.08734472, 0.14635094],
         [0.01171875, 0.04296875, 0.08463541, 0.

In [1]:
partials_

NameError: name 'partials_' is not defined

In [13]:
partials[-1].shape

(3, 5, 4)